In [ ]:
#import libraries

import os
import re
import pandas as pd
import numpy as np
import random
import itertools
import matplotlib.pyplot as pyp
import seaborn as sb
import nltk

#set working directory
os.chdir('K:\Specialemappe_XD1')

# csv file obtained via R 
raw_data = pd.read_csv('emneraw.csv', sep=',', encoding='latin1')
print(raw_data.columns)
                   
# find and remove NaN værdi
raw_data.isnull().sum()
np.where(raw_data['emne_fagbetegnelse'].isnull())[0]
na_list = raw_data[raw_data['emne_fagbetegnelse'].isnull()].index.tolist()
raw_data['emne_fagbetegnelse'][319147]
print(raw_data.iloc[319147, :])
print(raw_data.iloc[319148, :])
print(raw_data.iloc[574995, :])
print(raw_data.iloc[574996, :])
print(raw_data.iloc[992314, :])
print(raw_data.iloc[992315, :])

#drop those index
raw_data= raw_data.drop(index=[319147, 319148, 574995, 574996, 992314, 992315])

# fix encoding errors
raw_data['emne_fagbetegnelse'] = raw_data['emne_fagbetegnelse'].str.replace('Ã\x98','Ø')
raw_data['emne_fagbetegnelse'] = raw_data['emne_fagbetegnelse'].str.replace('Ã\x86','Æ')
raw_data['emne_fagbetegnelse'] = raw_data['emne_fagbetegnelse'].str.replace('Ã\x85','Å')
raw_data['emne_fagbetegnelse'] = raw_data['emne_fagbetegnelse'].str.replace('Â½','½')
raw_data['emne_fagbetegnelse'] = raw_data['emne_fagbetegnelse'].str.replace('Â§','§')

#drop useless column
raw_data=raw_data.drop(columns=['Unnamed: 0'])

#rename columns
raw_data = raw_data.rename(columns={
                         'emne_fagbetegnelse':'emne_fag_navn', 
                         'ant_elever':'antal_elever',
                         'ant_aktivitetid':'antal_gange_aktivitet'})


#### activities based on frequency

In [ ]:
raw_data.columns

#ascending order
df_sortedfq = raw_data.sort_values(by=['antal_gange_aktivitet'], ascending=False)
df_sortedfq.reset_index(inplace=True)

#   >5000 to millions - 2756 - 0.26%
fq1 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] >= 5000)]

#   >500 to 4999      - 61183 - 5.78%
fq2 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] >= 500) & (df_sortedfq['antal_gange_aktivitet'] <5000)]

#   >5 to 499        - 569340 - 53.80%
fq3 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] >= 5) & (df_sortedfq['antal_gange_aktivitet'] <499)]

#   >2 to 4          - 204794 - 19.35%
fq4 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] >= 2) & (df_sortedfq['antal_gange_aktivitet'] <5)]

#   1 time only      - 219878 - 20.78%
fq5 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] == 1)]

# snippet raw data in latex
table1= df_sortedfq.loc[100:150]
table1.columns
table1=table1.drop(columns=['index', 'oversaettelse'])
print(table1.to_latex(index=False)) 

In [ ]:
#visualization: length of unique activity names

# a) length raw strings
raw_list =df_sortedfq['emne_fag_navn'].tolist()

raw_length = []
for i in range(len(raw_list)):
    raw_length.append(len(raw_list[i]))

#vis a)
sb.set_theme(style="darkgrid")
ax_a= sb.displot(raw_length, binwidth=5, height=5)
pyp.title("Length - characters")
ax_a.fig.set_figwidth(5)
ax_a.fig.set_figheight(3)
ax_a.savefig('length_raw.png', dpi=800)

# b) length > 500 cleaned
fq_more500 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] >= 500)]
fq_more500_list = fq_more500['emne_fag_navn'].tolist()
fqmore500_length = [len(fq_more500_list[i]) for i in range(len(fq_more500_list))]
    
#vis b)
sb.set_theme(style="darkgrid")
ax_c= sb.displot(fqmore500_length, binwidth=5, height=5)
pyp.title("Length - most occurring activities >> 500")
ax_c.fig.set_figwidth(5)
ax_c.fig.set_figheight(3)
ax_c.savefig('length_more500.png', dpi=800)


# c) length  < 500 cleaned
fq_less500 = df_sortedfq.loc[(df_sortedfq['antal_gange_aktivitet'] <500)]
fq_less500_list = fq_less500['emne_fag_navn'].tolist()
fqless500_length = [len(fq_less500_list[i]) for i in range(len(fq_less500_list))]

#vis c)
sb.set_theme(style="darkgrid")
ax_d= sb.displot(fqless500_length, binwidth=5, height=5)
pyp.title("Length - less occurring activities < 500")
ax_d.fig.set_figwidth(5)
ax_d.fig.set_figheight(3)
ax_d.savefig('length_less500.png', dpi=800)
ax_d.plt.show()

#### unique activity names

In [ ]:
#unique abbreviations of activities registered by all schools
raw_list = raw_data['emne_fag_navn'].tolist()
unique_names = list(set(list_abbr))

In [ ]:
# visualization: how many potential short headlines per educational subject?

obl_fag = ['dansk', 'historie', 'matematik', 'billedkunst', 'biologi', 
           'engelsk','fransk', 'fysik_kemi', 'geografi', 'madkundskab',
           'håndværk_og_design', 'idræt', 'kristendomskundskab','musik',
           'natur_teknologi','samfundsfag','tysk']
length_fag = []  

raw_l =    [l.lower() for l in unique_names]

dan_l=      [label for label in raw_l if 'dan' in label] 
length_fag.append(len(dan_l))

his_l=      [label for label in raw_l if 'his' in label]
length_fag.append(len(his_l))

mat_l=      [label for label in raw_l if 'mat' in label]
length_fag.append(len(mat_l))

bilk_l=     [label for label in raw_l if 'bil' in label]
length_fag.append(len(bilk_l)) 

bio_l=      [label for label in raw_l if 'bio' in label]
length_fag.append(len(bio_l)) 

eng_l=      [label for label in raw_l if 'eng' in label]
length_fag.append(len(eng_l)) 

fr_l=       [label for label in raw_l if 'fra' in label]
length_fag.append(len(fr_l)) 

fykemi_l=   [label for label in raw_l if 'fy' in label]
length_fag.append(len(fykemi_l)) 

geo_l=     [label for label in raw_l if 'geo' in label]
length_fag.append(len(geo_l)) 

madk_l=    [label for label in raw_l if 'mad' in label]
length_fag.append(len(madk_l)) 

hdes_l=    [label for label in raw_l if 'hå' in label]
length_fag.append(len(hdes_l)) 

idr_l=      [label for label in raw_l if 'idr' in label]
length_fag.append(len(idr_l)) 

krins_l=    [label for label in raw_l if 'kri' in label]
length_fag.append(len(krins_l)) 

mus_l=      [label for label in raw_l if 'mus' in label]
length_fag.append(len(mus_l)) 

natek_l=  [label for label in raw_l if 'nat' in label]
length_fag.append(len(natek_l)) 

samf_l=     [label for label in raw_l if 'sam' in label]
length_fag.append(len(samf_l)) 

tys_l=     [label for label in raw_l if 'tys' in label]
length_fag.append(len(tys_l)) 

# merge fag and length by zip().  
list_tuples = list(zip(obl_fag, length_fag))  
print(list_tuples)  
  
# Converting lists of tuples into Dataframe.  
df_fag = pd.DataFrame(list_tuples, columns=['Fag', 'Number of instances'])  
print(df_fag)  
total_fag = df_fag['Number of instances'].sum()
#vis
pyp.figure(figsize=(8,12), dpi=1200)
ax_e=sb.barplot(y ="Fag",
                x ="Number of instances", 
                palette="Blues_d", orient= 'h',
                data = df_fag.sort_values(by=['Number of instances'], ascending=False))

pyp.title("No. of activities per subject name; total 48696, 20.7%")
pyp.savefig('subjectnames_bins.png')



In [ ]:
#example how to use pandas and concatenate / merge



# example for merging three different regex search and getting 'øvrige' examples
new_df = df.loc[(~df["emne_fag_navn"].str.contains('MAT'))
                          & (~df["emne_fag_navn"].str.contains('DAN'))
                          & (~df["emne_fag_navn"].str.contains('ENG'))]
sample_df = new_df.sample(n=3360, random_state=100)
sample_df['hånd_validering'] = ' '
sample_df.reset_index(inplace = True, drop = True)


#### how do we check a single activity name and contact a school?

In [ ]:
#all data entries for a specific abbrevations in str.contains('')

check_df = raw_data
#lowercasing for better view
check_df['emne_fag_navn']= c_df['emne_fag_navn'].apply(lambda x: x.lower())

single_check = check_df.loc[(raw_data['emne_fag_navn'].str.contains('DAN16B'))]

# using index 
check_df.loc[6644]

# count how many times is present (remember the redundancy of non-unique names)
check_df['emne_fag_navn'].str.contains('COVID').sum()

#identify school by institutionnummer on the institutionsregister
# https://data.stil.dk/InstregV2/Fremfind.aspx?InstNr=743002&SearchType=Inst

# Data: random sample of 52000 unique activity names

In [ ]:
#unique abbreviations of activities registered by all schools
list_activities = raw_data['emne_fag_navn'].tolist()
print(len(list_activities))
unique_activities = list(set(unique_activities))
print(len(unique_activities))

# df_unique_activities = pd.DataFrame(unique_activities, columns = ['emne_fag_navn'])
#print out excel data
# df_unique_activities.to_excel('df_211k_instances.xlsx')


# draw a random sample
# random.seed(1)
# random.shuffle(unique_abbr)
# sample_activities = random.sample(unique_activities, 52000)

# #dataframe and xlxs for labelling manually
# sample_df = pd.DataFrame(sample_activities, columns=['emne_fag_navn'])
# sample_df.reset_index(inplace=True)
# sample_df['set_label']= ' '

#print excel file for manual labelling
# sample_df.to_excel('sample52k.xlsx')


#### the sample file is renamed to "labelled_sample" once manual labelling is completed

In [ ]:
# verify that setting the labels is correct without mistakes
load_sample=pd.read_excel('sample52k.xlsx')
load_sample =load_sample.drop(columns=['Unnamed: 0'])
load_sample['set_label'].value_counts()

#### remaining unique activity names

In [ ]:
# unique activity names remaining from the 52k 
remaining_istances = [instance for instance in unique_abbr if instance not in abbr_52k_list]
print(len(remaining_istances))

#### draw additional 10 random samples from the remaining activity names

In [ ]:
#%% 159795 remaining unique names

#211795 unique abbreviations of activities registered by all schools
list_all = raw_data['emne_fag_navn'].tolist()
list_abbr = list(map(str, list_all))
unique_abbr = list(set(list_all))

# load unique abbreviation of activities from sample selected previously
load_sample=pd.read_excel('sample52k.xlsx')
load_sample.columns
load_sample['set_label'].value_counts()

#detect float element
# float_check = pd.to_numeric(load_sample['emne_fag_navn'],errors='coerce').notna()

all_abbr_sample52 = load_sample['emne_fag_navn'].tolist()
list_abbr_sample52 = list(map(str, all_abbr_sample52))
unique_abbr_sample52 =list(set(list_abbr_sample52))
len(unique_abbr_sample52) # 2 activities have been deleted since not recognized by schools

#filter name activities without those in the trained sample
filtered1 = [i for i in unique_abbr if i not in unique_abbr_sample52]
print(len(filtered1)) #159842

# draw 10 samples of 100 instances
random.seed(2)
random.shuffle(filtered1)
#1
sample_test1 = random.sample(filtered1,100)
#2
filtered2 =  [i for i in filtered1 if i not in sample_test1]
print(len(filtered2))
sample_test2 = random.sample(filtered2,100)
#3
filtered3 =  [i for i in filtered1 if i not in sample_test2]
print(len(filtered3))
sample_test3 = random.sample(filtered3,100)
#4
filtered4 =  [i for i in filtered3 if i not in sample_test3]
print(len(filtered4))
sample_test4 = random.sample(filtered4,100)
#5
filtered5 =  [i for i in filtered4 if i not in sample_test4]
print(len(filtered5))
sample_test5 = random.sample(filtered5,100)
#6
filtered6 =  [i for i in filtered5 if i not in sample_test5]
print(len(filtered6))
sample_test6 = random.sample(filtered6,100)
#7
filtered7 =  [i for i in filtered6 if i not in sample_test6]
print(len(filtered7))
sample_test7 = random.sample(filtered7,100)
#8
filtered8 =  [i for i in filtered7 if i not in sample_test7]
print(len(filtered8))
sample_test8 = random.sample(filtered8,100)
#9
filtered9 =  [i for i in filtered8 if i not in sample_test8]
print(len(filtered9))
sample_test9 = random.sample(filtered9,100)
#10
filtered10 =  [i for i in filtered9 if i not in sample_test9]
print(len(filtered10))
sample_test10 = random.sample(filtered10,100)

# #dataframe and xlxs for samples
#import itertools
list_samples = list(itertools.chain(sample_test1,sample_test2,sample_test3,sample_test4, sample_test5,\
                                    sample_test6,sample_test7, sample_test8, sample_test9, sample_test10))

samples_df = pd.DataFrame(list_samples, columns=['emne_fag_navn'])
samples_df.reset_index(inplace=True)
samples_df['set_label']= ' '

# samples_df.to_excel('10samplessave.xlsx')

# inspecting noise within the data 

In [ ]:
#dataframe with the only column of names
content = pd.DataFrame(update_data['emne_fagbetegnelse'])
content.shape
content.loc[50:70]
print(len(content.loc[50][0]))

#removing digits 0-9 that occur 1 or more times in the labels [column]
content['emne_fagbetegnelse'] = content['emne_fagbetegnelse'].str.replace('\d+', '')
content.loc[50:70]
print(len(content.loc[50][0]))

In [ ]:
#regex recap
strs = "how much for the maple syrup? $20.99? That's ricidulous!!!"
print (strs)
nstr = re.sub(r'[?|$|.|!]',r'',strs)
print (nstr)
nestr = re.sub(r'[^a-zA-Z0-9 ]',r'',nstr)
print (nestr)

import re
d=' ''3A LEJRSKOLE - NORMALTIMER /BO -.- .-. ./.V - MAT., ./. ÅRGANG 2022 ???? \\\\\\ $$$'
print(d)
d= re.sub('-', r'', d)
print(d)
d=re.sub(r'[?|$|.|!|\|/|]',r'',d)
print(d)
d = re.sub('\W+','', d)
print(d)

#### which kind of noise do we find?

In [ ]:
# dataframe with all instances including a specific noise
noise_check = load_sample.loc[(load_sample['emne_fag_navn'].str.contains('*'))]

#reporting found noise
#  " " , ' ', (), %, ,* . ,.., // , / , ?, ??, multiple??, @, _ ,  +, >, §§BD A, § space, .-., 
#  different format of ´´ -> ‘’EKSAMENSVAGT FP  ´, ¨SFO, ¨.Å, ` AH, ”MIG OG MIN ROBOT”, 
#  ½,  melleumru usage x4 
#- _  -- ---  - (BORTFALDER)
#( ) { } ambiguous count
#() 0{} 0[]
# [*]
#[´] [`] ["] [#] 6[@] [,] [;] [:] [.] [<>]
#! _?_ [?] \W+ ?
#[+-] 0~ 0[|]
# [0123456789] 130k: 0 21k, 1 31k, 2 18k, 3 14k, 4 13k, 5 11k, 6 10k, 7 9k, 8 8k, 9 6k

The usage of numbers, -, whitespaces is quite diverse. 

Noise and characters are close to or merget to meaningful abbreviations and words. 

It makes sense to replace with a whitespace all these characters and numbers 
to help tokenization.

If / replaced by whitespace we get two words or two characters or two abbreviations for fysik/kemi, natur/teknologi, håndværk/design.

In case of N/T makes sense to replace in natur teknologi (formal language)

#### use NLTK 3.6.5 instead of NLTK 3.6.6

#### verify what happens with tokenization

In [ ]:
content = df['emne_fag_navn'] #variable containing name activities

content_text=[] 
for name in content:                
    name = name.lower()               #lowercasing
    # a bug introduced in NLTK 3.6.6 > IndexError: list index out of range
    # needs to remove . ! ? any end-of-sentence characters
    name = name.replace("!", '')    #removing all !
    name = name.replace("?", '')    #removing all *
    name = name.replace(".", '')    #removing all *
    # name = name.replace(",", '')    #removing all ,
    name = nltk.word_tokenize(name, language= 'danish')
    
    for piece in name:
        if piece != ' ':
            content_text.append(piece)
print('Sample of the output')
print(content_text[0:5])
print(content_text[5:10])

def lexical_diversity(text):
    return len(set(text)) / len(text)

print("Number of words:", len(content_text))
print("Number of unique words:", len(set(content_text)))
print("Lexical diversity:", round(lexical_diversity(content_text), 4))

#collocations are expressions og multiple words which commonly co-occur, more often than singularly
bigrams = BigramCollocationFinder.from_words(content_text)  #bigrams 
collocation = bigrams.nbest(BigramAssocMeasures.likelihood_ratio, 50)
collocation

bigrams_freq = list(nltk.bigrams(content_text))
bigr_freq = nltk.FreqDist(bigrams_freq)
bigr_freq.plot(10)
print("Most frequent bigrams")

print("Right: most frequent bigrams; Left: most frequent collocations")
list(zip(collocation, bigr_freq)) #there´s difference between most freq collocations and bigrams
